In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers
from tensorflow.keras.datasets import mnist

In [2]:
num_classes = 10 # total classes (0-9 digits).

learning_rate = 0.001
training_steps = 200
batch_size = 128
display_step = 10

conv1_units= 32 # number of filters for 1st conv layer.
conv2_units = 64 # number of filters for 2nd conv layer.
fc1_units = 1024 # number of neurons for 1st fully-connected layer.

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train / 255., x_test / 255.

In [4]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [5]:
class ConvNet(Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = layers.Conv2D(conv1_units, kernel_size=5, activation=tf.nn.relu)
        self.maxpool1 = layers.MaxPool2D(2, strides=2)
        self.conv2 = layers.Conv2D(conv2_units, kernel_size=3, activation=tf.nn.relu)
        self.maxpool2 = layers.MaxPool2D(2, strides=2)
        
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(fc1_units)
        self.dropout = layers.Dropout(rate=0.5)
        self.out = layers.Dense(num_classes)
        
    def call(self, x, is_training=False):
        x = tf.reshape(x, [-1, 28, 28, 1])
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x, training=is_training)
        x = self.out(x)
        if not is_training:
            x = tf.nn.softmax(x)
        return x
    
conv_net = ConvNet()

In [6]:
def cross_entropy_loss(x, y):
    y = tf.cast(y, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    return tf.reduce_mean(loss)

def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

In [7]:
optimizer = tf.optimizers.Adam(learning_rate)

In [8]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = conv_net(x, is_training=True)
        loss = cross_entropy_loss(pred, y)
        
    gradients = g.gradient(loss, conv_net.trainable_variables)
    optimizer.apply_gradients(zip(gradients, conv_net.trainable_variables))

In [9]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = conv_net(batch_x)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 10, loss: 1.830245, accuracy: 0.781250
step: 20, loss: 1.623213, accuracy: 0.890625
step: 30, loss: 1.622952, accuracy: 0.875000
step: 40, loss: 1.554584, accuracy: 0.953125
step: 50, loss: 1.536372, accuracy: 0.960938
step: 60, loss: 1.540476, accuracy: 0.945312
step: 70, loss: 1.516587, accuracy: 0.968750
step: 80, loss: 1.536101, accuracy: 0.953125
step: 90, loss: 1.528477, accuracy: 0.960938
step: 100, loss: 1.528449, accuracy: 0.968750
step: 110, loss: 1.535256, accuracy: 0.929688
step: 120, loss: 1.505390, accuracy: 0.976562
step: 130, loss: 1.490353, accuracy: 0.984375
step: 140, loss: 1.503910, accuracy: 0.968750
step: 150, loss: 1.492171, accuracy: 0.984375
step: 160, loss: 1.481122, accuracy: 1.000000
step: 170, loss: 1.507161, accuracy: 0.968750
step: 180, loss: 1.505684, accuracy: 0.960938
step: 190, loss: 1.499278, accuracy: 0.984375
step: 200, loss: 1.503493, accuracy: 0.976562


In [10]:
pred = conv_net(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.976900
